In [ ]:
# SETUP_ARTIFACT_PATHS
from pathlib import Path
BASE_DIR = Path.cwd().resolve()
if BASE_DIR.name == 'notebooks':
    BASE_DIR = BASE_DIR.parent
MODELS_DIR = BASE_DIR / 'models'
MAPPINGS_DIR = BASE_DIR / 'mappings'
MODELS_DIR.mkdir(parents=True, exist_ok=True)
MAPPINGS_DIR.mkdir(parents=True, exist_ok=True)
MODELO_OUT = MODELS_DIR / 'modelo_char_lm_v2.keras'
MAPEAMENTOS_OUT = MAPPINGS_DIR / 'mapeamentos_v2.pkl'
print('Modelo:', MODELO_OUT)
print('Mapeamentos:', MAPEAMENTOS_OUT)

## Como salvar artefatos (guia rápido)
Use os caminhos padronizados definidos na célula de setup:
- Modelo: `MODELO_OUT` (Path)
- Mapeamentos: `MAPEAMENTOS_OUT` (Path)

Recomendação prática:
- Defina `NOME_ARQUIVO_MODELO = str(MODELO_OUT)` e `NOME_ARQUIVO_MAPS = str(MAPEAMENTOS_OUT)`
- Use essas variáveis nas chamadas de `model.save(...)` e ao salvar/abrir mapeamentos (`open(NOME_ARQUIVO_MAPS, 'wb')`).

# LLM v2 — Char-level Text Generator (PT-BR)

Melhorias principais:
- Embeddings de caracteres (em vez de one-hot).
- 2 camadas LSTM maiores com dropout.
- Callbacks (EarlyStopping, ReduceLROnPlateau, ModelCheckpoint).
- Amostragem com temperatura + top-k/top-p para saída mais fluida.
- Treino a partir de um arquivo local (evita downloads).

In [1]:
import os, sys, pickle, random
from pathlib import Path
import numpy as np
import tensorflow as tf

# Configurações principais
URL_LIVRO = "https://www.gutenberg.org/files/55752/55752-0.txt"
NOME_ARQUIVO_MODELO = str(MODELO_OUT)
NOME_ARQUIVO_MAPS = str(MAPEAMENTOS_OUT)

SEED = 42
SEQ_LEN = 160
BATCH_SIZE = 128
EPOCHS = 30
EMBED_DIM = 128
LSTM_UNITS = 512
DROPOUT = 0.2
RECURRENT_DROPOUT = 0.1  # cuidado: pode reduzir performance em GPU
LR = 2e-3

# Semente para reprodutibilidade
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

d:\Dropbox\Coding\TCC\.venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Dropbox\Coding\TCC\.venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Dropbox\Coding\TCC\.venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violat

In [2]:
# Baixar os dados
ARQUIVO_TEXTO = tf.keras.utils.get_file(
    "dom_casmurro.txt", URL_LIVRO
)

# Carregar texto (UTF-8) e preparar vocabulário de caracteres
assert Path(ARQUIVO_TEXTO).exists(), (
    f'Arquivo {ARQUIVO_TEXTO} não encontrado. Coloque seu corpus local e rode novamente.'
)
with open(ARQUIVO_TEXTO, 'rb') as f:
    texto = f.read().decode('utf-8', errors='ignore').lower()

# Opcional: limpeza simples (remover controles)
texto = ''.join(ch for ch in texto if ch == '\n' or 32 <= ord(ch) <= 126 or ord(ch) >= 128)
print('Tamanho do corpus:', len(texto))

caracteres = sorted(list(set(texto)))
char_to_id = {c: i for i, c in enumerate(caracteres)}
id_to_char = {i: c for i, c in enumerate(caracteres)}
vocab_size = len(caracteres)
print('Tamanho do vocabulário:', vocab_size)

# Codificar texto em IDs
ids = np.fromiter((char_to_id[c] for c in texto), dtype=np.int32)
len_ids = ids.shape[0]
print('IDs gerados:', len_ids)

Tamanho do corpus: 381556
Tamanho do vocabulário: 67
IDs gerados: 381556


In [3]:
# Criar dataset com tf.data
ds = tf.data.Dataset.from_tensor_slices(ids)
ds = ds.window(SEQ_LEN + 1, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda w: w.batch(SEQ_LEN + 1))
def split_input_target(chunk):
    return chunk[:-1], chunk[1:]
ds = ds.map(split_input_target, num_parallel_calls=tf.data.AUTOTUNE)
ds = ds.shuffle(buffer_size=min(10000, len_ids)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int32, name=None), TensorSpec(shape=(None, None), dtype=tf.int32, name=None))>

In [4]:
# Construir modelo com Embedding + 2x LSTM
from tensorflow.keras import layers, optimizers, callbacks, models

def build_model(vocab_size):
    m = models.Sequential([
        layers.Input(shape=(None,), dtype='int32'),
        layers.Embedding(vocab_size, EMBED_DIM),
        layers.LSTM(LSTM_UNITS, return_sequences=True, dropout=DROPOUT, recurrent_dropout=RECURRENT_DROPOUT),
        layers.LSTM(LSTM_UNITS, return_sequences=True, dropout=DROPOUT, recurrent_dropout=RECURRENT_DROPOUT),
        layers.Dense(vocab_size, activation='softmax'),
    ])
    opt = optimizers.Adam(learning_rate=LR, clipnorm=1.0)
    m.compile(optimizer=opt, loss='sparse_categorical_crossentropy')
    return m

if Path(NOME_ARQUIVO_MODELO).exists() and Path(NOME_ARQUIVO_MAPS).exists():
    print('>> Carregando modelo v2 existente...')
    model = tf.keras.models.load_model(NOME_ARQUIVO_MODELO)
    with open(NOME_ARQUIVO_MAPS, 'rb') as f:
        maps = pickle.load(f)
        char_to_id = maps['char_to_id']
        id_to_char = maps['id_to_char']
    vocab_size = len(char_to_id)
    # NÃO gere novo vocabulário local!
else:
    print('>> Criando novo modelo v2...')
    caracteres = sorted(list(set(texto)))
    char_to_id = {c: i for i, c in enumerate(caracteres)}
    id_to_char = {i: c for i, c in enumerate(caracteres)}
    vocab_size = len(caracteres)
    model = build_model(vocab_size)

model.summary()

>> Carregando modelo v2 existente...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 128)      │         8,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 512)      │     1,312,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, None, 512)      │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 67)       │        34,371 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,364,747 (39.54 MB)

 Trainable params: 3,454,915 (13.18 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,909,832 (26.36 MB)

In [4]:
# Treinamento com callbacks (pule esta célula se já possui um modelo salvo)
cb = [
    callbacks.ModelCheckpoint(NOME_ARQUIVO_MODELO, save_best_only=True, monitor='loss', mode='min'),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=5e-5),
    callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True),
]

history = model.fit(ds, epochs=EPOCHS, callbacks=cb)

# Salvar mapeamentos
with open(NOME_ARQUIVO_MAPS, 'wb') as f:
    pickle.dump({'char_to_id': char_to_id, 'id_to_char': id_to_char}, f)
print('Treino concluído e artefatos salvos.')

NameError: name 'callbacks' is not defined

In [7]:
# Amostragem: temperatura + top-k + top-p (nucleus)
def sample_next(prob, temperature=1.0, top_k=0, top_p=0.0):
    prob = np.asarray(prob).astype('float64')
    if prob.ndim != 1:
        raise ValueError("prob must be 1D")
    # temperatura
    prob = np.log(prob + 1e-9) / max(temperature, 1e-6)
    prob = np.exp(prob)
    prob = prob / prob.sum()

    n = prob.shape[0]

    # top-k (limitar k ao tamanho do vocabulário)
    if top_k and top_k > 0:
        k = min(int(top_k), n)
        if k < n:
            idx = np.argpartition(prob, -k)[-k:]
            mask = np.zeros_like(prob)
            mask[idx] = prob[idx]
            denom = mask.sum()
            if denom > 0:
                prob = mask / denom

    # top-p (nucleus)
    if top_p and top_p > 0.0:
        sort_idx = np.argsort(prob)[::-1]
        sorted_prob = prob[sort_idx]
        cumsum = np.cumsum(sorted_prob)
        cutoff = np.searchsorted(cumsum, top_p) + 1
        idx_keep = sort_idx[:cutoff]
        mask = np.zeros_like(prob)
        mask[idx_keep] = prob[idx_keep]
        denom = mask.sum()
        if denom > 0:
            prob = mask / denom

    return np.random.choice(n, p=prob)


def gerar_texto_v2(model, char_to_id, id_to_char, prompt, n_chars=400, temperature=0.9, top_k=50, top_p=0.9):
    prompt = (prompt or '').lower()
    prompt = ''.join(c for c in prompt if c in char_to_id)
    if not prompt:
        prompt = ' ' if ' ' in char_to_id else list(char_to_id.keys())[0]

    context_ids = [char_to_id[c] for c in prompt][-SEQ_LEN:]
    out = [c for c in prompt]

    for _ in range(n_chars):
        x = np.array([context_ids], dtype=np.int32)
        preds = model.predict(x, verbose=0)
        # pegar distribuição do último passo (protege contra saída com return_sequences)
        if preds.ndim == 3:
            probs = preds[0, -1]
        elif preds.ndim == 2:
            probs = preds[0]
        else:
            raise RuntimeError("Formato inesperado de saída do modelo")
        next_id = sample_next(probs, temperature=temperature, top_k=top_k, top_p=top_p)
        next_ch = id_to_char.get(int(next_id), '')
        out.append(next_ch)
        context_ids.append(int(next_id))
        if len(context_ids) > SEQ_LEN:
            context_ids = context_ids[-SEQ_LEN:]
    return ''.join(out)

print('Funções de geração prontas.')

Funções de geração prontas.


In [12]:
# Exemplo rápido de uso (ajuste os parâmetros a gosto)
if 'model' in globals():
    texto = gerar_texto_v2(
        model, char_to_id, id_to_char,
        prompt='se eu contratasse uma pessoa para me ajudar a escrever',
        n_chars=500, temperature=0.5, top_k=40, top_p=0.9
    )
    print("--- TEXTO GERADO (v2) ---")
    print(texto)
else:
    print('Modelo não carregado/criado. Execute as células anteriores.')


--- TEXTO GERADO (v2) ---
se eu contratasse uma pessoa para me ajudar a escrever nella a deus e ao diabo.
apenas que falei desbem falar no cerebro, é provavel que a ideia
não batesse as azas senão pela necessidade que sentia do vir ao ar e
á vida. a vida é tão bella que a mesma ideia da morte precisa de vir
primeiro a ella, antes de se ver cumprida. já me vás entendendo; lê
agora outro capitulo.




cxxxiv

o dia de sabbado.

a ideia saiu finalmente do cerebro. era noite, e não pude dormir, por
mais que a sacudisse de mim. tambem nenhuma noite me passou tão curta.
amanheceu
